In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
sns.set(font='SimHei')  #seaborn作图默认不支持中文，需要额外设置
%matplotlib inline
%config InlineBackend.figure_format="retina"


In [ ]:
train = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv')
test = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')
train.info()
test.info()


数据没有空值，可以直接使用。但没有空值不代表没有异常。

In [ ]:
#查看数据情况
train.head()

In [ ]:
train['count'].describe()


先从数值型数据入手，可以看出租赁数量（count）数值差异大，所以希望观察一下它们的密度分布
均值191，标准差181，50%分位数是145，75%分位数是284，最大值977，说明右侧存在长尾。

In [ ]:
#观察数密度分布
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
fig.set_size_inches(6,5)

sns.distplot(train['count'])

ax.set(xlabel='count',title='Distribution of count',)



分布偏斜比较严重，所以希望能把这一列数据的异常值处理一下，首先排除掉3个标准差以外的数据

In [ ]:
#把超出3倍标准差的数据，共147个剔除
train = train.loc[np.abs(train['count']-train['count'].mean()) < (3*train['count'].std())]
#对剔除异常值后的count和count_log进行比较（因为count中最小为1，故此处np.log不需要加1来避免log0的出现）
train['count_log'] = np.log(train['count'])
f, [ax1, ax2] = plt.subplots(1,2, figsize=(15,6))
sns.distplot(train['count'], ax=ax1)
ax1.set_title('Distribution of count')
sns.distplot(train['count_log'], ax=ax2)
ax2.set_title('Distribution of count_log')


经过对数变换后数据分布更均匀，大小差异也缩小了，使用这样的标签对训练模型是有益的。

In [ ]:
train['casual'].describe()

In [ ]:
train['registered'].describe()

In [ ]:
train['_data'] = 'train'
test['_data'] = 'test'
#首先合并数据
df = train.append(test, ignore_index=True)
#整理列顺序
df = pd.DataFrame(df, columns=train.columns)
df.tail()

In [ ]:
#将它们对数化，但因为其最小值为0，所以我们要加1防止log0的出现
df['casual_log'] = np.log(df['casual']+1)
df['registered_log'] = np.log(df['registered']+1)

In [ ]:
#转化格式
dt = pd.DatetimeIndex(df['datetime'])
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')
#细化，这里去了weekday，方便看每周的变化情况
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour
df['weekday'] = df['datetime'].dt.weekday


In [ ]:
f, ax = plt.subplots(2,2, figsize=(12,10))
sns.distplot(df.temp, ax=ax[0,0])
ax[0,0].set_title('Distribution of temp')
sns.distplot(df.atemp, ax=ax[0,1])
ax[0,1].set_title('Distribution of atemp')
sns.distplot(df.humidity, ax=ax[1,0])
ax[1,0].set_title('Distribution of humidity')
sns.distplot(df.windspeed, ax=ax[1,1])
ax[1,1].set_title('Distribution of windspeed')


通过这个分布可以发现一些问题，比如风速为什么0的数据很多，而观察统计描述发现空缺值在1--6之间，

从这里似乎可以推测，数据本身或许是有缺失值的，但是用0来填充了，

但这些风速为0的数据会对预测产生干扰，希望使用随机森林根据相同的年份，月份，季节，温度，湿度等几个特征来填充一下风速的缺失值。

In [ ]:
wind_0 = df[df['windspeed']==0]
wind_not0 = df[df['windspeed']!=0]
y_label = wind_not0['windspeed']
#猜测风速和天气以及时间都有关
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
windcolunms = ['season', 'weather', 'temp', 'atemp', 'humidity', 'hour', 'month']
model.fit(wind_not0[windcolunms], y_label.astype('int'))
pred_y = model.predict(wind_0[windcolunms])
#预测结果填充
wind_0['windspeed'] = pred_y
df_rfw = wind_not0.append(wind_0)
df_rfw.reset_index(inplace=True)
df_rfw.head()


In [ ]:
df_rfw = df_rfw.drop('index', axis=1)
#查看处理后的风速情况
f, ax = plt.subplots(figsize=(8,5))
sns.distplot(df_rfw['windspeed'], ax=ax)
ax.set_title('Distribution of handled windspeed')

In [ ]:
df['windspeed'].describe()


In [ ]:
df_rfw['windspeed'].describe()

通过随机森林填充处理好了

In [ ]:
#查看各组数据和count的相关性
f, ax = plt.subplots(figsize=(20,16))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(df_rfw[df_rfw['count'].notnull()].corr(),linewidths=0.5,vmax=.8,square=True,linecolor='w',cmap=cmap, ax=ax, annot=True, lw=.1)


count和temp/atemp/hour有较明显正相关，和humidity有较明显负相关

In [ ]:
#整体看一下租赁额相关的三个值和其他特征值的关系。
sns.pairplot(df_rfw ,x_vars=['holiday','workingday','weather','season','weekday','hour','windspeed','humidity','temp','atemp'] ,
             y_vars=['casual','registered','count'] , plot_kws={'alpha': 0.1})

放大部分来看看

In [ ]:
# Regression Plots with respect to Temperature, Humidity and Windspeed
fig = plt.figure(figsize=(18, 8))
axes = fig.add_subplot(1, 3, 1)
sns.regplot(data=df_rfw, x='temp', y='count',ax=axes)
axes.set(title='Reg Plot for temp vs. count')
axes = fig.add_subplot(1, 3, 2)
sns.regplot(data=df_rfw, x='humidity', y='count',ax=axes, color='r')
axes.set(title='Reg Plot for Humidity vs. count')
axes = fig.add_subplot(1, 3, 3)
sns.regplot(data=df_rfw, x='windspeed', y='count',ax=axes, color='g')
axes.set(title='Reg Plot for Windspeed vs. count')
plt.show()

1.会员在工作日出行多，节假日出行少，临时用户则相反；
2.一季度出行人数总体偏少；
3.租赁数量随天气等级上升而减少；
4.小时数对租赁情况影响明显，会员呈现两个高峰，非会员呈现一个正态分布；
5.租赁数量随风速增大而减少；
6.温度、湿度对非会员影响比较大，对会员影响较小

In [ ]:
#看一下总的
corrDf = df_rfw.corr() 
corrDf

In [ ]:
#相关性排序
df_rfw[df_rfw['count'].notnull()].corr()['count'].sort_values(ascending=False)

可以看出特征值对租赁数量的影响力度为,时段>温度>湿度>年份>月份>季节>天气等级>风速>星期几>是否工作日>是否假日

In [ ]:
count_columns = ['count', 'registered', 'casual']
f, [ax1, ax2] = plt.subplots(1,2, figsize=(15,5))
df_rfw.groupby(['year','season'])[count_columns].mean().plot.line(ax=ax1)
df_rfw.groupby('season')[count_columns].mean().plot.line(ax=ax2)
ax2.set_xticks(range(1,5))

整体可以看出registered和casual的用户均呈现上升的趋势，夏季用户租赁量增速最快，冬天租赁量回退，春秋租赁量缓慢增加。
可以看出无论是临时用户还是会员用户用车的数量都在秋季迎来高峰，而春季度用户数量最低

In [ ]:
f, ax = plt.subplots(figsize=(10,5))
df_rfw.groupby('hour')[count_columns].mean().plot.line(ax=ax)
ax.set_title('租车量在一天内的变化', fontsize=15)

registered用户存在明显的早晚高峰，对应上班时间，中午午饭时间也有小峰值，casual用户则是在10-20点有灵活租赁。

In [ ]:
f, [ax1,ax2] = plt.subplots(1,2,figsize=(15,5))
df_rfw_w = df_rfw.loc[df_rfw['workingday']==1]
df_rfw_notw = df_rfw.loc[df_rfw['workingday']==0]
df_rfw_w.groupby('hour')[count_columns].mean().plot.line(ax=ax1)
df_rfw_notw.groupby('hour')[count_columns].mean().plot.line(ax=ax2)
ax1.set_title('工作日租车量在一天内的变化', fontsize=15)
ax2.set_title('非工作日租车量在一天内的变化', fontsize=15)


将工作日与非工作日分开看，可以看到在工作日有更明显的早晚高峰，在非工作日两种用户的租赁趋势相同。

1.工作日对于会员用户上下班时间是两个用车高峰，而中午也会有一个小高峰，猜测可能是外出午餐的人； 

2.而对临时用户起伏比较平缓，高峰期在17点左右；

3.并且会员用户的用车数量远超过临时用户。

4.对非工作日而言租赁数量随时间呈现一个正态分布，高峰在14点左右，低谷在4点左右，且分布比较均匀。

# 我们再来看详细一点的

In [ ]:
fig, axes = plt.subplots(nrows=1,ncols=1)
fig.set_size_inches(12, 10)
sns.boxplot(data=df_rfw_w,y="count",x="hour",orient="v")
axes.set(xlabel='hour', ylabel='Count',title="工作日租车量在一天内的变化")


可以发现工作日有双高峰，一个是 8 点，另外两个是 17 -18 点。按照上班群体的角度看，正好是上下班的早晚高峰。

In [ ]:
fig, axes = plt.subplots(nrows=1,ncols=1)
fig.set_size_inches(12, 10)
sns.boxplot(data=df_rfw_notw,y="count",x="hour",orient="v")
axes.set(xlabel='hour', ylabel='Count',title="非工作日租车量在一天内的变化")


可以发现非工作日有高峰， 在10 -19 点，是出行游玩的时期。

In [ ]:
#我们多设一个高峰的特征，判断此时间段是否为常规高峰期
df_rfw['peak'] = df_rfw[['hour', 'workingday']].apply(lambda x: (0, 1)[(x['workingday'] == 1 and  ( x['hour'] == 8 or 17 <= x['hour'] <= 18 )) or (x['workingday'] == 0 and  10 <= x['hour'] <= 19)], axis = 1)

In [ ]:
f, ax = plt.subplots(figsize=(10,5))
df_rfw.groupby('weekday')[count_columns].mean().plot.line(ax=ax)
ax.set_title('租赁数量的周变化曲线')

通过周变化曲线，发现registered用户在周一到周五的租赁数量稳定，周末两天减少20%，casual用户在周末租赁数量是工作日的两倍

In [ ]:
weather_df = df_rfw.groupby('weather', as_index=True).agg({'casual':'mean', 'registered':'mean'})
weather_df.plot.bar(stacked=True,title = 'Average number of rentals initiated per hour in different weather')


天气条件越不好，租赁人数越少，和预期相同，但是天气为4，也就是暴雨暴雪时反而租赁量很高，需要具体查看分析。

In [ ]:
df_rfw.loc[df_rfw['weather']==4]


天气情况为4时，只有一组数据，而且是周一的18点，是一天的租赁最高峰，相比正常情况下400+的租赁，只有164，说明天气对租赁量影响是很大的

In [ ]:
f, [ax1, ax2] = plt.subplots(2,1,figsize=(12,10))
df_rfw.groupby('temp')[count_columns].mean().plot.line(ax=ax1)
ax1.set_title('租赁数量随气温的变化曲线')
df_rfw.groupby('atemp')[count_columns].mean().plot.line(ax=ax2)
ax2.set_title('租赁数量随体感气温的变化曲线')

租赁数量随温度升高，呈上升趋势，气温36℃左右，体感温度40℃左右达到最大值。且体感温度和气温的曲线走势接近，只是延后4℃。

气温38℃以上有异常，需根据具体数据确定

In [ ]:
df_rfw.loc[df_rfw['temp']>=38].head(10)

在2012-07-07一天11:00-18:00存在长时间高租赁，可能有集体活动，具有偶然性

In [ ]:
#因为temp和atemp相关系数为0.99，将temp和atemp合并
#df_rfw['new_temp'] = (df_rfw['temp'] + df_rfw['atemp'])/2
#改用PCA将其合并
from sklearn.decomposition import PCA
pca=PCA(n_components=1)
pca.fit(df_rfw[['temp', 'atemp']])

print(f" Variance explained after PCA : {pca.explained_variance_ratio_}")

pca算出其相关性很高

将其合并

In [ ]:
from scipy.stats import norm
df_rfw['temperature']=pca.fit_transform(df_rfw[['temp','atemp']])
sns.distplot(df_rfw['temperature'], fit=norm)

In [ ]:

#湿度的影响
f, ax = plt.subplots(figsize=(10,5))
df_rfw.groupby('humidity')[count_columns].mean().plot.line(ax=ax)
ax.set_title('租赁数量随湿度的变化曲线')

和预期类似，可以观察到在湿度20左右租赁数量迅速达到高峰值，此后缓慢递减。随湿度的增加，租赁量缓慢下降。

In [ ]:
#风速跨度比较大，所以对其进行分组
df_rfw['wind_class'] = pd.cut(df_rfw['windspeed'], 9)
f, [ax1, ax2] = plt.subplots(2,1,figsize=(12,10))
df_rfw.groupby('windspeed')[count_columns].mean().plot.line(ax=ax1)
ax1.set_title('租赁数量随风速的变化曲线')
df_rfw.groupby('wind_class')[count_columns].mean().plot.line(ax=ax2)
ax2.set_title('租赁数量随风速等级的变化曲线')

风速较低时，对租赁量的影响不大，风速超过45时，租赁量迅速缩小，和异常天气时的情况类似。

查看风速>51的数据，均值偏高可以归于异常值。

In [ ]:
df_rfw.loc[df_rfw['windspeed']>=51]

# 我们再来观察一下租车数量在各年每月的变换

In [ ]:
sns.pointplot(data=df_rfw, x='month', y=df_rfw['count'],  hue='year')

可以看到2012年每个月的租车量都比2011年高，所以说明年份是很关键的特征

In [ ]:
df_rfw.info()

In [ ]:
df_rfw['ideal'] = df_rfw[['temperature', 'windspeed']].apply(lambda x: (0, 1)[x['temperature'] > 27 and x['windspeed'] < 30], axis = 1)
df_rfw['sticky'] = df_rfw[['humidity', 'workingday']].apply(lambda x: (0, 1)[x['workingday'] == 1 and x['humidity'] >= 60], axis = 1)

对分类数据进行one-hot编码(由于CART决策树使用二分类，所以将多类别型数据使用one-hot转化成多个二分型类别)

In [ ]:
#season_dummy=pd.get_dummies(df_rfw['season'],prefix='season')
#weather_dummy=pd.get_dummies(df_rfw['weather'],prefix='weather')
#month_dummy=pd.get_dummies(df_rfw['month'],prefix='month')
#df_rfw1 = pd.concat([df_rfw,season_dummy,weather_dummy,month_dummy],axis=1)
#df_rfw1.head()


编写构建模型所用的相关函数

In [ ]:

# #

def get_rmsle(y_pred, y_actual):#计算误差评估
    diff = np.log(y_pred + 1) - np.log(y_actual + 1)
    mean_error = np.square(diff).mean()
    return np.sqrt(mean_error)


def get_data():#获取训练集数据
    data = df_rfw[df_rfw['_data'] == 'train'].copy()
    return data


def custom_train_test_split(data, cutoff_day=15):#划分训练集测试集(将数据拆分为要素和目标标注)
    train = data[data['day'] <= cutoff_day]
    test = data[data['day'] > cutoff_day]

    return train, test


def prep_data(data, input_cols):
    X = data[input_cols]
    y_r = data['registered_log']
    y_c = data['casual_log']

    return X, y_r, y_c


def predict_on_validation_set(model, input_cols):
    data = get_data()

    train, test = custom_train_test_split(data)

    X_train, y_train_r, y_train_c = prep_data(train, input_cols)
    X_test, y_test_r, y_test_c = prep_data(test, input_cols)

    model_r = model.fit(X_train, y_train_r)
    y_pred_r = np.exp(model_r.predict(X_test)) - 1

    model_c = model.fit(X_train, y_train_c)
    y_pred_c = np.exp(model_c.predict(X_test)) - 1

    y_pred_comb = np.round(y_pred_r + y_pred_c)
    y_pred_comb[y_pred_comb < 0] = 0

    y_test_comb = np.exp(y_test_r) + np.exp(y_test_c) - 2

    score = get_rmsle(y_pred_comb, y_test_comb)
    return (y_pred_comb, y_test_comb, score)

df_test = df_rfw[df_rfw['_data'] == 'test'].copy()

# 在测试集上进行预测，并从对数尺度转换输出
def predict_on_test_set(model, x_cols):
    # prepare training set
    df_train = df_rfw[df_rfw['_data'] == 'train'].copy()
    X_train = df_train[x_cols]
    y_train_cas = df_train['casual_log']
    y_train_reg = df_train['registered_log']

    # prepare test set
    X_test = df_test[x_cols]

    casual_model = model.fit(X_train, y_train_cas)
    y_pred_cas = casual_model.predict(X_test)
    y_pred_cas = np.exp(y_pred_cas) - 1
    registered_model = model.fit(X_train, y_train_reg)
    y_pred_reg = registered_model.predict(X_test)
    y_pred_reg = np.exp(y_pred_reg) - 1
    # add casual & registered predictions together
    return y_pred_cas + y_pred_reg


采用随机森林和GBM模型，但各自选取的特征值不同。

In [ ]:
#from sklearn.ensemble import RandomForestRegressor
#model =RandomForestRegressor(n_estimators=1000,random_state=42)
#model.fit(df_train,count_log)
#pred=model.predict(df_test)
# random forest model
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
params = {'n_estimators': 1000, 'max_depth': 15, 'random_state': 0, 'min_samples_split' : 5, 'n_jobs': -1}
rf_model = RandomForestRegressor(**params)
rf_cols = [
    'weather', 'temperature', 'windspeed',
    'workingday', 'season', 'holiday', 
    'hour',   'peak'
]
rf_p, rf_t, rf_score = predict_on_validation_set(rf_model, rf_cols)
print(rf_score)



# GBM model
params = {'n_estimators': 150, 'max_depth': 5, 'random_state': 0, 'min_samples_leaf' : 10, 'learning_rate': 0.1, 'subsample': 0.7, 'loss': 'ls'}
gbm_model = GradientBoostingRegressor(**params)
gbm_cols = [
    'weather', 'temperature', 'humidity', 'windspeed',
    'holiday', 'workingday', 'season',
    'hour', 'year'
    ]


(gbm_p, gbm_t, gbm_score) = predict_on_validation_set(gbm_model, gbm_cols)
print(gbm_score)

# the blend gives a better score on the leaderboard, even though it does not on the validation set
y_p = np.round(.2*rf_p + .8*gbm_p)
print(get_rmsle(y_p, rf_t))

In [ ]:
# predctions on test dataset
rf_pred = predict_on_test_set(rf_model, rf_cols)
gbm_pred = predict_on_test_set(gbm_model, gbm_cols)

In [ ]:
# 获取两个模型输出的加权平均值
y_pred = np.round(.20*rf_pred + .80*gbm_pred)

In [ ]:
# output predictions for submission
df_test['count'] = y_pred
final_df = df_test[['datetime', 'count']].copy()
final_df.to_csv('submission.csv', index=False)